<a href="https://colab.research.google.com/github/m-urena/bison/blob/main/Tactical_Growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import smtplib
from datetime import datetime, timedelta
from email.mime.text import MIMEText
from apscheduler.schedulers.blocking import BlockingScheduler
from google.colab import userdata

def get_spy_data(end_date=None):
    if end_date is None:
        end_date = datetime.today()
    # Always get data for one year
    start_date_one_year = end_date - timedelta(days=365)
    data = yf.download('ES=F', start=start_date_one_year.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'), auto_adjust=True)['Close']

    return data

def get_latest_hourly_futures_price(end_date=None):
    if end_date is None:
        end_date = datetime.now() # Use datetime.now() for hourly data

    # Fetch data for the last 5 days to ensure we capture the latest hourly price
    start_date_hourly = end_date - timedelta(days=5)

    # Download hourly data for ES=F
    hourly_data = yf.download(
        'ES=F',
        start=start_date_hourly,
        end=end_date,
        interval='1h',
        auto_adjust=True
    )

    if not hourly_data.empty:
        # Extract 'Close' prices and return the latest available
        return hourly_data['Close'].iloc[-1]
    else:
        return None

def calculate_moving_averages(prices):
    ma150 = prices.rolling(window=150).mean()
    ma30 = prices.rolling(window=30).mean()
    return ma150, ma30

def max_drawdown(prices):
    roll_max = prices.cummax()
    daily_drawdown = prices / roll_max - 1.0
    max_dd = daily_drawdown.min()
    return max_dd

def get_signal_for_specific_date(full_prices_data, date_to_evaluate):
    # Filter prices to only include data up to the date_to_evaluate
    prices_up_to_date = full_prices_data.loc[full_prices_data.index <= date_to_evaluate.strftime('%Y-%m-%d')]

    if prices_up_to_date.empty or len(prices_up_to_date) < 150:
        return "Not enough data", 0, None, None, None, date_to_evaluate # Return date_to_evaluate for context

    # Get the actual last available trading day's data within this filtered set
    latest_price_date_for_evaluation = prices_up_to_date.index[-1]
    today_price = prices_up_to_date.iloc[-1]

    ma150, ma30 = calculate_moving_averages(prices_up_to_date)

    # Drop NaNs to get valid MA values for the latest available date
    ma150 = ma150.dropna()
    ma30 = ma30.dropna()

    if ma150.empty or ma30.empty or ma150.index[-1] != latest_price_date_for_evaluation:
        return "Not enough valid MA data", 0, None, None, None, latest_price_date_for_evaluation

    ma150_today = ma150.iloc[-1]
    ma30_today = ma30.iloc[-1]

    # Ensure scalar values for comparison
    if isinstance(today_price, pd.Series):
        today_price = today_price.item()
    if isinstance(ma150_today, pd.Series):
        ma150_today = ma150_today.item()
    if isinstance(ma30_today, pd.Series):
        ma30_today = ma30_today.item()

    # Medium and Short term risk signals
    medium_term_signal = 'positive' if today_price > ma150_today else 'negative'
    short_term_signal = 'positive' if today_price > ma30_today else 'negative'

    # Combined Risk Signal
    if medium_term_signal == 'negative' and short_term_signal == 'negative':
        risk_score = 1
        risk_label = 'risk off'
    elif medium_term_signal == 'positive' and short_term_signal == 'positive':
        risk_score = 3
        risk_label = 'risk on'
    else:
        risk_score = 2
        risk_label = 'risk neutral'

    # Volatility Override
    mdd = max_drawdown(prices_up_to_date)
    # Ensure mdd is a scalar for comparison
    if isinstance(mdd, pd.Series):
        mdd = mdd.item()

    if mdd < -0.24 and short_term_signal == 'positive':
        risk_score = 3
        risk_label = 'risk on (volatility override)'

    return risk_label, risk_score, today_price, ma150_today, ma30_today, latest_price_date_for_evaluation

def send_email(subject, body, to_email, from_email, smtp_server, smtp_port, smtp_user, smtp_password):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls() # Use STARTTLS for port 587
        server.login(smtp_user, smtp_password)
        server.sendmail(from_email, [to_email], msg.as_string())

def scheduled_email_job():
    # Get current time for email subject and 'Data as of' for hourly price
    current_datetime = datetime.now()
    today_date_str = current_datetime.strftime('%Y-%m-%d')

    # Get all required daily data once for efficiency (for MAs and Yesterday's Signal)
    full_prices = get_spy_data(end_date=current_datetime)
    current_hourly_price = get_latest_hourly_futures_price(end_date=current_datetime)

    # Initialize variables for Today's Signal
    risk_label_today = "Not enough data for Today's Signal"
    current_price_for_display = None
    ma150_for_display = None
    ma30_for_display = None
    data_as_of_today = current_datetime

    # Calculate MAs based on the full daily data up to the latest daily trading day
    ma150_full, ma30_full = calculate_moving_averages(full_prices)

    ma150_latest_daily = ma150_full.iloc[-1] if not ma150_full.empty else None
    ma30_latest_daily = ma30_full.iloc[-1] if not ma30_full.empty else None

    # Check if we have enough daily data and valid MAs for Today's Signal calculation
    if current_hourly_price is not None and ma150_latest_daily is not None and ma30_latest_daily is not None:
        # Ensure scalar values for comparison
        if isinstance(current_hourly_price, pd.Series):
            current_hourly_price = current_hourly_price.item()
        if isinstance(ma150_latest_daily, pd.Series):
            ma150_latest_daily = ma150_latest_daily.item()
        if isinstance(ma30_latest_daily, pd.Series):
            ma30_latest_daily = ma30_latest_daily.item()

        # Use hourly price for current price, but daily MAs
        current_price_for_display = current_hourly_price
        ma150_for_display = ma150_latest_daily
        ma30_for_display = ma30_latest_daily

        # Medium and Short term risk signals for today
        medium_term_signal_today = 'positive' if current_hourly_price > ma150_latest_daily else 'negative'
        short_term_signal_today = 'positive' if current_hourly_price > ma30_latest_daily else 'negative'

        # Combined Risk Signal for today
        if medium_term_signal_today == 'negative' and short_term_signal_today == 'negative':
            risk_label_today = 'risk off'
        elif medium_term_signal_today == 'positive' and short_term_signal_today == 'positive':
            risk_label_today = 'risk on'
        else:
            risk_label_today = 'risk neutral'

        # Volatility Override for today - uses daily MDD
        mdd_today = max_drawdown(full_prices)
        if isinstance(mdd_today, pd.Series):
            mdd_today = mdd_today.item()

        if mdd_today < -0.24 and short_term_signal_today == 'positive':
            risk_label_today = 'risk on (volatility override)'

    # --- Calculate Yesterday's Signal using the existing daily logic ---
    yesterday_result = ("Not enough trading days", 0, None, None, None, None) # Default
    if len(full_prices.index) >= 2:
        actual_second_latest_trading_day = full_prices.index[-2]
        yesterday_result = get_signal_for_specific_date(full_prices, actual_second_latest_trading_day)

    subject = f"[{today_date_str}] Tactical Growth Signal Update"
    body = f"Daily Tactical Growth Signal Update\n\n"

    # --- Build Today's Signal part of the email body ---
    body += f"--- Today's Signal ({today_date_str}) ---\n"
    if risk_label_today == "Not enough data for Today's Signal":
        body += f"Signal: {risk_label_today.upper()}\n"
    else:
        body += f"Data as of: {data_as_of_today.strftime('%Y-%m-%d %H:%M')}\n"
        body += f"Signal: {risk_label_today.upper()}\n"
        body += f"Current Futures Price: {current_price_for_display:.2f}\n"
        body += f"150-day MA: {ma150_for_display:.2f}\n"
        body += f"30-day MA: {ma30_for_display:.2f}\n"

    # --- Build Yesterday's Signal part of the email body ---
    body += f"\n--- Yesterday's Signal (Based on {today_date_str} - 1 trading day) ---\n"
    if yesterday_result[0] in ["Not enough data", "Not enough trading days", "Not enough valid MA data"]:
        body += f"Signal: {yesterday_result[0].upper()}\n"
    else:
        risk_label_yesterday, _, _, _, _, date_yesterday = yesterday_result
        body += f"Data as of: {date_yesterday.strftime('%Y-%m-%d')}\n"
        body += f"Signal: {risk_label_yesterday.upper()}\n"

    # --- Comparison ---
    body += "\n--- Comparison ---\n"
    # Need to compare today's (hourly based) signal with yesterday's (daily based) signal
    if risk_label_today != "Not enough data for Today's Signal" and \
       yesterday_result[0] not in ["Not enough data", "Not enough trading days", "Not enough valid MA data"]:
        # Compare the core signal labels, ignoring volatility override text for comparison
        today_core_label = risk_label_today.split('(')[0].strip()
        yesterday_core_label = yesterday_result[0].split('(')[0].strip()
        if today_core_label != yesterday_core_label:
            body += f"Signal changed from {yesterday_result[0].upper()} to {risk_label_today.upper()}.\n"
        else:
            body += f"Signal remained {risk_label_today.upper()}.\n"
    else:
        body += "Unable to compare signals due to insufficient data for one or both days.\n"

    # Change these accordingly
    to_email = 'murena@bisonwealth.com'
    from_email = 'murena@bisonwealth.com'
    smtp_server = 'smtp.office365.com'
    smtp_port = 587
    smtp_user = userdata.get('SMTP_USER')
    smtp_password = userdata.get('SMTP_PASSWORD')
    send_email(subject, body, to_email, from_email, smtp_server, smtp_port, smtp_user, smtp_password)



In [ ]:
scheduled_email_job()
print("Email job initiated. Please check your inbox for the test email.")